In [1]:
import pandas as pd
import os
import xml.etree.ElementTree as ET
import io
import warnings

In [ ]:
import urllib.request
import json
from Data.data import api_ids
import datetime

URL = "https://data.gov.il/api/3/action/datastore_search?resource_id="

for year in range(2023, 2010, -1):
    url = f"{URL}{api_ids[str(year)]}"
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read())

    records = data['result']['records']
    df = pd.DataFrame.from_records(records)
    print(f"Finished\t{year}!\t{df.shape}")
    if year==2014:
        break

In [ ]:
import requests
import xml.etree.ElementTree as ET
import io

# URL of the XML file
url = "https://data.gov.il/dataset/pensia-net/resource/88f86a79-bdbe-4afd-9e8b-592da9607207/download/pension-2001.xml"
# url = "https://data.gov.il/dataset/pensia-net/resource/136d60f7-adab-43e2-9744-2eb1c265cfee/download/pension-2011.xml"
# url = "https://data.gov.il/dataset/pensia-net/resource/953c98df-295b-4de2-9488-45ef43e943d7/download/pension-2012.xml"
# url = "https://data.gov.il/dataset/pensia-net/resource/75cf7f21-5e69-4518-8d8c-5b18460e61f3/download/pension-2013.xml"
# url = "https://data.gov.il/dataset/pensia-net/resource/4de0e578-d03c-444c-874e-11f45e668c8d/download/pension-2014.xml"

# Send a GET request to fetch the XML file
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    xml_content = "<ROWSET>" + response.text[53:]
#     # Define the file path where the XML will be saved
#     file_path = '.\Data\pension-2012.xml'
#     # Write the XML content to a file
#     with open(file_path, 'w') as file:
#         file.write(xml_content)
#     result = "XML file successfully saved."
else:
    result = f"Failed to download the XML file. Status code: {response.status_code}"
    
tree = ET.parse(io.StringIO(xml_content))
root = tree.getroot()

valid_tags = ["ID_KRN", "TKF_DIVUACH", "TSUA_NOMINALI_BFOAL"]
data = []
for row in root.findall('ROW'):
    data_dict = {}
    for child in row:
        if child.tag in valid_tags:
            data_dict[child.tag] = child.text
    data.append(data_dict)

df = pd.DataFrame(data)
df.shape